In [67]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from datetime import datetime
from babel.dates import format_date
import seaborn as sns

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder



from pprint import pprint


In [24]:
# nom de fichier et chemin relatif
filename = 'merge_raw_metrics_dataset.csv'
path = '../data/metrics/'
# création d'un dataframe à partir du csv de données
df = pd.read_csv(
    path+filename, index_col=0).sort_values(by='created_at', ascending=True)
df.head(2)


C:\Users\DeOliveira\AppData\Local\Temp\ipykernel_18740\505030999.py:5: DtypeWarning: Columns (10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


,id,sn,name,type,generation,name_counter,value,name_op,level,source,message,timestamp,criticality,identification,created_at,varnishLevelsTargetvolume,varnishLevelsTotalvolume
0,4169748,NaN,Print Engine 1,Varnish Printer,NaN,3D Varnish Counter,1792992,Viktor,Operator,NaN,NaN,NaN,NaN,NaN,2022-04-15 05:55:06.678000,36192.322612,100000
1,4169748,NaN,iFoil L,iFoil,Gen. 2,Total Pages Counter,22881,Viktor,Operator,NaN,NaN,NaN,NaN,NaN,2022-04-15 05:55:06.678000,36192.322612,100000


In [65]:
# Convert 'timestamp' column to datetime type
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Sort dataframe by 'timestamp'
df = df.sort_values(by='timestamp')
df.head(5)

,id,name,type,generation,name_counter,value,name_op,level,source,message,timestamp,criticality,identification,created_at,varnishLevelsTargetvolume,varnishLevelsTotalvolume
3,4169749,Print Engine 1,Varnish Printer,NaN,3D Varnish Counter,1792992,Viktor,Operator,PLC,JV-Ti non prêt : impression impossible,2022-04-15 05:55:23.462000+00:00,INFO,391,2022-04-15 05:55:06.829000,36192.322612,100000
4,4169749,iFoil L,iFoil,Gen. 2,Total Pages Counter,22881,Viktor,Operator,PLC,JV-Ti non prêt : impression impossible,2022-04-15 05:55:23.462000+00:00,INFO,391,2022-04-15 05:55:06.829000,36192.322612,100000
5,4169749,iFoil L,iFoil,Gen. 2,Foiled Pages Counter,31092,Viktor,Operator,PLC,JV-Ti non prêt : impression impossible,2022-04-15 05:55:23.462000+00:00,INFO,391,2022-04-15 05:55:06.829000,36192.322612,100000
214,4170152,iFoil L,iFoil,Gen. 2,Foiled Pages Counter,31092,Viktor,Operator,iFoil,JV-Ti non prêt : impression impossible,2022-04-15 06:06:56.278000+00:00,INFO,391,2022-04-15 06:06:35.404000,36192.322612,100000
210,4170152,Print Engine 1,Varnish Printer,NaN,3D Varnish Counter,1792992,Viktor,Operator,iFoil,JV-Ti non prêt : impression impossible,2022-04-15 06:06:56.278000+00:00,INFO,391,2022-04-15 06:06:35.404000,36192.322612,100000


In [25]:
df.columns

Index(['id', 'sn', 'name', 'type', 'generation', 'name_counter', 'value',
       'name_op', 'level', 'source', 'message', 'timestamp', 'criticality',
       'identification', 'created_at', 'varnishLevelsTargetvolume',
       'varnishLevelsTotalvolume'],
      dtype='object')

In [26]:
df.shape

(3546276, 17)

In [27]:
null_columns = df.isna().any()
print("Columns with missing values:")
print(null_columns[null_columns == True])


Columns with missing values:
sn                True
generation        True
source            True
message           True
timestamp         True
criticality       True
identification    True
dtype: bool


In [28]:
# cela va supprimer la colonne sn
df.dropna(axis='columns', how='all', inplace=True)

In [58]:
null_values_count = df.isna().sum()
null_values_count

id                                 0
name                               0
type                               0
generation                   1182092
name_counter                       0
value                              0
name_op                            0
level                              0
source                       3302193
message                      3302244
timestamp                    3302193
criticality                  3302193
identification               3302193
created_at                         0
varnishLevelsTargetvolume          0
varnishLevelsTotalvolume           0
dtype: int64

In [64]:
for column, value in null_values_count.items():
    if value > 0:
        print(f"{column}: {value} valeurs manquantes sur {df.shape[0]}")


generation: 1182092 valeurs manquantes sur 3546276
source: 3302193 valeurs manquantes sur 3546276
message: 3302244 valeurs manquantes sur 3546276
timestamp: 3302193 valeurs manquantes sur 3546276
criticality: 3302193 valeurs manquantes sur 3546276
identification: 3302193 valeurs manquantes sur 3546276


In [29]:
df.shape

(3546276, 16)

In [32]:
df['criticality'].value_counts()

INFO       179610
WARNING     34497
ERROR       29976
Name: criticality, dtype: int64

In [33]:
#  identify categorical columns
categorical_columns = df.select_dtypes(include=['object', 'category']).columns

categorical_columns

Index(['name', 'type', 'generation', 'name_counter', 'name_op', 'level',
       'source', 'message', 'timestamp', 'criticality', 'identification',
       'created_at'],
      dtype='object')

In [68]:
# Initialize OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

# Perform one-hot encoding on categorical columns
one_hot_encoded_columns = ohe.fit_transform(df[categorical_columns])

# Create a new DataFrame with the one-hot encoded columns and the appropriate column names
one_hot_encoded_df = pd.DataFrame(
    one_hot_encoded_columns,
    columns=ohe.get_feature_names_out(categorical_columns),
    index=df.index
)

# Merge the one-hot encoded columns with the original dataframe and drop the original categorical columns
df = pd.concat([df.drop(categorical_columns, axis=1),
               one_hot_encoded_df], axis=1)


c:\Users\DeOliveira\Documents\Fabeon\MGI\Prediction-Erreur-JetVarnish3D\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


KeyboardInterrupt: 

In [66]:
# Label encode the target variable 'criticality'
le_criticality = LabelEncoder()
df['criticality'] = le_criticality.fit_transform(df['criticality'])
